In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.runtime_logging import start
from neuron.agents import UserAgent
from neuron.agents.agents import LearnerAgent, KnowledgeRepresenterAgent
from neuron.agents.llm_agents import PerceiverAgent
from neuron.scenarios import RoundRobin, RoundRobinManager

from neuron.capabilities.perceiver import SemanticTemplateFillerCapability

# Start logging
logging_session_id = start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

c:\Users\FS-Ma\miniconda3\envs\neuron\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logging session ID: 644985c9-9f0c-4a10-a14f-00623fe24d5c


In [3]:
import pandas as pd
df = pd.read_json("C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/data/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
# read the data 
for business in df:
    business = business
    break
business.drop(['business_id'], axis=1, inplace=True)

In [4]:
llm_config={
    "config_list": [{
        "client": "groq",
        "model": "llama3-groq-70b-8192-tool-use-preview",
        "api_key": "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"
        }
]}

user_agent = UserAgent(
    name="User Agent",
    llm_config=llm_config,
)

perceiver_agent = PerceiverAgent(
    llm_config=llm_config,
)

leaner_agent = LearnerAgent(
    llm_config = {
    "config_list": [{
        "client": "Begreat",
        "model": "distilgpt2",
        "model_dir": "C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/models/be_great_yelp",
    }]},
)

#knowledge_representer_agent = KnowledgeRepresenterAgent(
#    system_message="You are responsible to generate representations of the information.",
#    llm_config= {
#        "config_list": [{
#            "client": "LLMEmbedding",
#            "usage_type ": "embeddings", # // or "embeddings", "classification", etc.
#            "model_dir": "C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/models/erasmo_yelp_gpt2-medium_60_True",
#            "model_name": "gpt2-medium"
#    }]},
#)


semantic_template_filler = SemanticTemplateFillerCapability(row=business.iloc[0], llm_config=llm_config)
semantic_template_filler.add_to_agent(perceiver_agent)

groupchat = RoundRobin(agents=[user_agent, perceiver_agent, leaner_agent], messages=[])
manager = RoundRobinManager(groupchat=groupchat, llm_config=None)

user_agent.initiate_chat(
    manager, message="I like sushi e meu nome é sildofo e meus reviews_count são 10 eu sou gosto de restaurantes com 10 estrelas."
)

c:\Users\FS-Ma\miniconda3\envs\neuron\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\FS-Ma\miniconda3\envs\neuron\lib\site-packages\be_great\great.py:487: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary object

User Agent (to chat_manager):

I like sushi e meu nome é sildofo e meus reviews_count são 10 eu sou gosto de restaurantes com 10 estrelas.

--------------------------------------------------------------------------------
perceiver_agent (to chat_manager):

{
    "name": ["sildofo"],
    "address": [np.nan],
    "city": [np.nan],
    "state": [np.nan],
    "postal_code": [np.nan],
    "latitude": [np.nan],
    "longitude": [np.nan],
    "stars": ["10"],
    "review_count": ["10"],
    "is_open": [np.nan],
    "attributes": [np.nan],
    "categories": [np.nan],
    "hours": [np.nan]
}

--------------------------------------------------------------------------------


100%|██████████| 1/1 [00:06<00:00,  6.59s/it]

learner_agent (to chat_manager):

name is sildofo, address is 5275 W Sunset Blvd, city is Las Vegas, state is placeholder, postal_code is 89031, latitude is 36.1238804, longitude is -115.122655, stars is 10.0, review_count is 10.0, is_open is 1.0, attributes is placeholder, categories is Shopping, hours is {'Monday': '11:0-17:0'

--------------------------------------------------------------------------------
